# How to handle connections to devices

## Specifying connection parameters

By default, connections are handled automatically based on the information in the inventory.  
Here is an example of `hosts.yaml` with simple and advanced connection details:

In [1]:
%run handling_connections/example1/display_inventory.py

```yaml
---
simple_host:
    hostname: simple-host.my-lab.com
    username: simple-user
    password: simple-password
    platform: cisco_ios

advanced_host:
    # standard parameters that will be re-used across different connections
    username: my_user
    password: my_password
    platform: linux
    napalm_options:
        # standard parameters override upper level and will be used only for the napalm connection
        # missing parameters are read from the top level
        port: 443
        platform: eos
        connection_options:
            # non-standard library-specific options supported by the backend library
            optional_args:
                eos_autoComplete: true
    netmiko_options:
        platform: arista_eos
        connection_options:
            global_delay: 2
    netmiko_telnet_options: # non-default connection name
        plugin: netmiko  # in this case plugin option is mandatory
        # terminal server details
        hostname: 1.2.3.4
        port: 3001
        platform: arista_eos_telnet
```

There are 5 reserved variable names that are used by Nornir connection handling:
- `hostname`: hostname or IP address
- `username`
- `password`
- `port`: if not specified, the default port chosen by the backend library will be used
- `platform`: a connection plugin passes this value with a key supported by the backend library. For example, `netmiko` library uses `device_type`, so netmiko connection plugin will use `platform` value as `device_type` in netmiko. Refer to the backend library documentation to specify this parameter correctly.

You can specify these parameters on the host level, but you can also override default parameters on the connection level.  
Let's consider `napalm_options`:  
`platform` field was overwritten and is now `eos` instead of `linux`, and the `port` was specified too.  
You can also specify non-standard library specific details under `connection_options`.  
For example, during connection establishment NAPALM accept an argument `optional_args`, while `netmiko` accepts an extra option `global_delay`.

The **key name** for the connection specific options is important. In this example, we have `napalm_options`, `netmiko_options`, `netmiko_telnet_options`.
The format of this key is `<connection_name>_options`

Let's consider the first two and we will get back to the third one in a minute.  
At the moment of this writing, Nornir has 3 connection-oriented plugins:
- **NAPALM** with the default connection name "napalm"
- **Netmiko** with the default connection name "netmiko"
- **Paramiko** with the default connection name "paramiko"

The default "registration" can also be found in the file `nornir/plugins/connections/__init__.py`
These are default connection names which are needed to be used in the inventory if you would like to load these options automatically during the corresponding task execution.  

This means that you can have only **one** connection established per a **single** backend library. Most of the times it is enough, but there are situations when you may need to have multiple connections established using the same backend library.  
For example, you would like to establish SSH and Telnet (via terminal server) connections using `netmiko` plugin.
This is why you can specify **arbitrary** connection name as a key, for example `netmiko_telnet_options`. Because a non-default name for the connection is specified, there are 2 constraints:
1. It is required to specify a plugin using the key `plugin`. For example, `plugin: netmiko`
2. During the task execution you need to provide a new connection name using `conn_name` argument. For example: `conn_name="netmiko_telnet"` (see an example below)

Let's see how the connections are handled in the code, first automatically, then manually.  
Here is the inventory used for the next examples:

In [2]:
%run handling_connections/example2/display_inventory.py

```yaml
---
rtr00:
    platform: mock
    napalm_options:  # uses plugin name "napalm" as a connection name
        connection_options:
            optional_args:
                path: handling_connections/example2/mocked_data
    secondary_options:  # uses non-default connection name "secondary"
        plugin: napalm  # the name of a registered plugin is "napalm"
        connection_options:
            optional_args:
                path: handling_connections/example2/mocked_data

```

## Automatic connection handling

In [3]:
from nornir.core import InitNornir
from nornir.plugins.functions.text import print_result
from nornir.plugins.tasks.networking import napalm_get

In [4]:
with InitNornir(config_file="handling_connections/example2/config.yaml") as nr:
    rtr = nr.filter(name="rtr00")
    r = rtr.run(
        task=napalm_get,
        getters=["facts"]
    )  # connection will be opened here and stored with "napalm" key
    # it will be re-used if another task uses napalm
    print_result(r)
# all opened connections are closed here, where context manager block is done

napalm_get**********************************************************************
* rtr00 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': [ 'Ethernet1',
                                 'Ethernet2',
                                 'Ethernet3',
                                 'Ethernet4',
                                 'Management1'],
             'model': 'vEOS',
             'os_version': '4.15.5M-3054042.4155M',
             'serial_number': '',
             'uptime': '...',
             'vendor': 'Arista'}}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


Alternatively, you can close all connections explicitly:

In [5]:
nr = InitNornir(config_file="handling_connections/example2/config.yaml")
rtr = nr.filter(name="rtr00")
r = rtr.run(
    task=napalm_get,
    getters=["facts"]
)  # connection will be opened here and stored with "napalm" key
# it will be re-used if another tasks use napalm
print_result(r)
nr.close_connections()  # all opened connections are closed here explicitly

napalm_get**********************************************************************
* rtr00 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': [ 'Ethernet1',
                                 'Ethernet2',
                                 'Ethernet3',
                                 'Ethernet4',
                                 'Management1'],
             'model': 'vEOS',
             'os_version': '4.15.5M-3054042.4155M',
             'serial_number': '',
             'uptime': '...',
             'vendor': 'Arista'}}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


### What if I need several connections using the same plugin which was discussed above?  

In [6]:
with InitNornir(config_file="handling_connections/example2/config.yaml") as nr:
    rtr = nr.filter(name="rtr00")
    r = rtr.run(
        task=napalm_get,
        conn_name="secondary",  # conn_name is specified
        getters=["facts"]
    )  # connection will be opened here and stored with "secondary" key
    # it will be re-used if another task uses connection "secondary"
    print_result(r)
# all opened connections are closed here

napalm_get**********************************************************************
* rtr00 ** changed : False *****************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': [ 'Ethernet1',
                                 'Ethernet2',
                                 'Ethernet3',
                                 'Ethernet4',
                                 'Management1'],
             'model': 'vEOS',
             'os_version': '4.15.5M-3054042.4155M',
             'serial_number': '',
             'uptime': '...',
             'vendor': 'Arista'}}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## Manual connection handling

In some circumstances, you may want to manage connections manually.  
To do so you can use 
[open_connection](../ref/api/inventory.rst#nornir.core.inventory.Host.open_connection), [close_connection](../ref/api/inventory.rst#nornir.core.inventory.Host.close_connection), [close_connections](../ref/api/inventory.rst#nornir.core.inventory.Host.close_connections) and [Nornir.close_connections](../ref/api/nornir.rst#nornir.core.Nornir.close_connections). For instance:

In [7]:
def task_manages_connection_manually(task):
    task.host.open_connection("napalm")
    r = task.run(
        task=napalm_get,
        getters=["facts"]
    )
    task.host.close_connection("napalm")
    
nr = InitNornir(config_file="handling_connections/example2/config.yaml")
rtr = nr.filter(name="rtr00")
r = rtr.run(
    task=task_manages_connection_manually,
)
print_result(r)

task_manages_connection_manually************************************************
* rtr00 ** changed : False *****************************************************
vvvv task_manages_connection_manually ** changed : False vvvvvvvvvvvvvvvvvvvvvvv INFO
---- napalm_get ** changed : False --------------------------------------------- INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': [ 'Ethernet1',
                                 'Ethernet2',
                                 'Ethernet3',
                                 'Ethernet4',
                                 'Management1'],
             'model': 'vEOS',
             'os_version': '4.15.5M-3054042.4155M',
             'serial_number': '',
             'uptime': '...',
             'vendor': 'Arista'}}
^^^^ END task_manages_connection_manually ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


or when you need two connections using the same plugin:

In [8]:
def task_manages_multiple_connections_manually(task):
    task.host.open_connection("napalm")
    task.host.open_connection("secondary", plugin="napalm")
    r1 = task.run(
        task=napalm_get,
        getters=["facts"]
    )
    r2 = task.run(
        task=napalm_get,
        conn_name="secondary",
        getters=["facts"]
    )
    task.host.close_connection("secondary")
    task.host.close_connection("napalm")
    
nr = InitNornir(config_file="handling_connections/example2/config.yaml")
rtr = nr.filter(name="rtr00")
r = rtr.run(
    task=task_manages_multiple_connections_manually,
)
print_result(r)

task_manages_multiple_connections_manually**************************************
* rtr00 ** changed : False *****************************************************
vvvv task_manages_multiple_connections_manually ** changed : False vvvvvvvvvvvvv INFO
---- napalm_get ** changed : False --------------------------------------------- INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface_list': [ 'Ethernet1',
                                 'Ethernet2',
                                 'Ethernet3',
                                 'Ethernet4',
                                 'Management1'],
             'model': 'vEOS',
             'os_version': '4.15.5M-3054042.4155M',
             'serial_number': '',
             'uptime': '...',
             'vendor': 'Arista'}}
---- napalm_get ** changed : False --------------------------------------------- INFO
{ 'facts': { 'fqdn': 'localhost',
             'hostname': 'localhost',
             'interface